In [53]:
import pandas as pd
import json

In [108]:
order_data = pd.read_csv("Datasets/order_data.csv")
customer_data = pd.read_csv("Datasets/customer_data.csv")
store_data = pd.read_csv("Datasets/store_data.csv")
test_data_question = pd.read_csv("Datasets/test_data_question.csv")

# ORDER DATA 

In [55]:
order_data.head(10)

,CUSTOMER_ID,STORE_NUMBER,ORDER_CREATED_DATE,ORDER_ID,ORDERS,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME
0,362204699,2156,2024-07-24,7247194287,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
1,269612955,1419,2025-02-15,791214421,"{""orders"": [{""item_details"": [{""item_name"": ""R...",Digital,WWT,ToGo
2,585330633,2249,2025-02-15,7575285208,"{""orders"": [{""item_details"": [{""item_name"": ""2...",Digital,WWT,ToGo
3,950661333,2513,2024-03-29,4253875716,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
4,434985772,1754,2024-04-08,7150407872,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
5,126084616,949,2025-02-15,3060989630,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,ToGo
6,426992703,2156,2025-02-15,1655782790,"{""orders"": [{""item_details"": [{""item_name"": ""1...",Digital,WWT,ToGo
7,772568272,820,2025-02-15,4015163280,"{""orders"": [{""item_details"": [{""item_name"": ""O...",Digital,WWT,Delivery
8,715522716,1161,2024-06-08,3057756219,"{""orders"": [{""item_details"": [{""item_name"": ""F...",Digital,WWT,ToGo
9,591908874,820,2025-02-15,4349588916,"{""orders"": [{""item_details"": [{""item_name"": ""R...",Digital,WWT,ToGo


In [56]:
print(f"order_data shape: {order_data.shape}")

order_data shape: (1414410, 8)


In [57]:
print(order_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1414410 entries, 0 to 1414409
Data columns (total 8 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   CUSTOMER_ID            1414410 non-null  int64 
 1   STORE_NUMBER           1414410 non-null  int64 
 2   ORDER_CREATED_DATE     1414410 non-null  object
 3   ORDER_ID               1414410 non-null  int64 
 4   ORDERS                 1414410 non-null  object
 5   ORDER_CHANNEL_NAME     1414410 non-null  object
 6   ORDER_SUBCHANNEL_NAME  1414410 non-null  object
 7   ORDER_OCCASION_NAME    1414410 non-null  object
dtypes: int64(3), object(5)
memory usage: 86.3+ MB
None


In [58]:
print(order_data.isnull().sum())

CUSTOMER_ID              0
STORE_NUMBER             0
ORDER_CREATED_DATE       0
ORDER_ID                 0
ORDERS                   0
ORDER_CHANNEL_NAME       0
ORDER_SUBCHANNEL_NAME    0
ORDER_OCCASION_NAME      0
dtype: int64


In [59]:
print(order_data.isnull().sum().sum())

0


In [60]:
print(order_data.duplicated().sum())

0


In [61]:
print(order_data.nunique())

CUSTOMER_ID               563346
STORE_NUMBER                  38
ORDER_CREATED_DATE           446
ORDER_ID                 1414410
ORDERS                    735527
ORDER_CHANNEL_NAME             1
ORDER_SUBCHANNEL_NAME          2
ORDER_OCCASION_NAME            2
dtype: int64


In [62]:
def clean_orders_column(order_json_string: str) -> str:
    filter_keywords = ("Order Memo", "Order Blankline")
    try:
        order_data = json.loads(order_json_string)
        original_items = order_data.get("orders", [{}])[0].get("item_details", [])
        filtered_items = [
            item
            for item in original_items
            if not item.get("item_name", "").startswith(filter_keywords)
        ]
        cleaned_order_data = {"orders": [{"item_details": filtered_items}]}
        return json.dumps(cleaned_order_data)

    except (json.JSONDecodeError, IndexError, AttributeError) as e:
        print(
            f"Could not process row due to error: {e}. Value: {order_json_string[:100]}"
        )
        return json.dumps({"orders": [{"item_details": []}]})


order_data_cleaned = order_data.copy()
order_data_cleaned["ORDERS"] = order_data_cleaned["ORDERS"].apply(clean_orders_column)


In [63]:
order_data["ORDERS"][3]

'{"orders": [{"item_details": [{"item_name": "Order Memo Item", "item_price": 0, "item_quantity": 5}, {"item_name": "Order Memo Paid", "item_price": 0, "item_quantity": 1}, {"item_name": "20 pc Grilled Wings", "item_price": 26.59, "item_quantity": 1}, {"item_name": "Order Memo ASAP", "item_price": 0, "item_quantity": 2}, {"item_name": "Ranch Dip - Regular", "item_price": 1.49, "item_quantity": 1}, {"item_name": "Order Blankline 2", "item_price": 0, "item_quantity": 1}, {"item_name": "Order Blankline 1", "item_price": 0, "item_quantity": 1}]}]}'

In [64]:
order_data_cleaned["ORDERS"][3]

'{"orders": [{"item_details": [{"item_name": "20 pc Grilled Wings", "item_price": 26.59, "item_quantity": 1}, {"item_name": "Ranch Dip - Regular", "item_price": 1.49, "item_quantity": 1}]}]}'

In [90]:
order_data_cleaned.to_csv("Datasets/order_data_cleaned.csv", index=False)

In [91]:
# always clean up resources to avoid over memory consumption (NOT EVERY ONE HAS 64 GB RAM :D)
del order_data_cleaned

# CUSTOMER DATA

In [109]:
customer_data.head(10)

,CUSTOMER_ID,CUSTOMER_TYPE
0,362204699,Registered
1,269612955,Registered
2,585330633,Guest
3,950661333,Registered
4,434985772,Guest
5,126084616,Registered
6,426992703,Guest
7,772568272,Registered
8,715522716,Guest
9,591908874,Registered


In [110]:
print(f"customer_data shape: {customer_data.shape}")

customer_data shape: (563346, 2)


In [111]:
print(customer_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563346 entries, 0 to 563345
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   CUSTOMER_ID    563346 non-null  int64 
 1   CUSTOMER_TYPE  563319 non-null  object
dtypes: int64(1), object(1)
memory usage: 8.6+ MB
None


In [112]:
print(customer_data.isnull().sum())

CUSTOMER_ID       0
CUSTOMER_TYPE    27
dtype: int64


In [113]:
print(f"Missing values in customer data :{customer_data.isnull().sum().sum()}")

Missing values in customer data :27


In [114]:
print(customer_data.duplicated().sum())

0


In [115]:
print(customer_data.nunique())

CUSTOMER_ID      563346
CUSTOMER_TYPE         5
dtype: int64


In [116]:
customer_data_cleaned = customer_data.copy()
customer_data_cleaned.fillna({"CUSTOMER_TYPE": "Guest"}, inplace=True)
customer_data_cleaned.to_csv("Datasets/customer_data_cleaned.csv", index=False)

In [119]:
print(customer_data_cleaned["CUSTOMER_TYPE"].isnull().sum())
del customer_data_cleaned

0


# STORE DATA

In [73]:
store_data.head(10)

,STORE_NUMBER,CITY,STATE,POSTAL_CODE
0,2156,GRAPEVINE,TX,76051
1,1419,HUNTERSVILLE,NC,28078
2,2249,NaN,NaN,32792
3,2513,LAS VEGAS,NV,89129
4,1754,ARDMORE,OK,73401
5,949,NaN,NaN,93033
6,820,NaN,NaN,37203
7,1161,NaN,NaN,78723-2429
8,2517,RUTHERFORD,CA,94573
9,4391,OMAHA,NE,68107


In [74]:
print(f"store_data shape: {store_data.shape}")

store_data shape: (38, 4)


In [75]:
print(store_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STORE_NUMBER  38 non-null     int64 
 1   CITY          25 non-null     object
 2   STATE         24 non-null     object
 3   POSTAL_CODE   36 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.3+ KB
None


In [76]:
print(store_data.isnull().sum())

STORE_NUMBER     0
CITY            13
STATE           14
POSTAL_CODE      2
dtype: int64


In [77]:
print(f"Missing values in store data :{store_data.isnull().sum().sum()}")

Missing values in store data :29


In [78]:
print(store_data.duplicated().sum())

0


In [79]:
print(store_data.nunique())

STORE_NUMBER    38
CITY            21
STATE           10
POSTAL_CODE     36
dtype: int64


In [136]:
# print(store_data["CITY"].values)
# print(store_data["STATE"].values)


In [139]:
store_data_cleaned = store_data.copy()
# idx_null = store_data[store_data["CITY"].isnull()].index
# store_data["CITY"][idx_null]
# idx_null = store_data[store_data["STATE"].isnull()].index
# store_data["STATE"][idx_null]
store_data_cleaned.fillna({"CITY": "Unknown", "STATE": "Unknown"}, inplace=True)
store_data_cleaned.to_csv("Datasets/store_data_cleaned.csv", index=False)
del store_data_cleaned

# TEST DATA QUESTION

In [80]:
test_data_question.head(10)

,CUSTOMER_ID,STORE_NUMBER,ORDER_ID,ORDER_CHANNEL_NAME,ORDER_SUBCHANNEL_NAME,ORDER_OCCASION_NAME,CUSTOMER_TYPE,item1,item2,item3
0,997177535,4915,9351345556,Digital,WWT,ToGo,Guest,Chicken Sub Combo,Ranch Dip - Regular,10 pc Spicy Wings Combo
1,345593831,949,3595377080,Digital,WWT,ToGo,Registered,Regular Buffalo Fries,10 pc Spicy Wings,3 pc Crispy Strips Combo
2,160955031,2249,4071757785,Digital,WWT,ToGo,Guest,Large Buffalo Fries,10 pc Spicy Wings,Ranch Dip - Regular
3,890671991,4154,3931766769,Digital,WWT,ToGo,Guest,6 pc Grilled Wings Combo,20 pc Grilled Wings,Fried Corn - Large
4,73989021,4094,3739700809,Digital,WWT,ToGo,Registered,Regular Buffalo Fries,20 pc Grilled Wings,Ranch Dip - Large
5,965629024,2734,1160484742,Digital,WWT,ToGo,Guest,3 pc Crispy Strips Combo,Chicken Sub Combo,Chicken Sub
6,554043002,430,3507129215,Digital,WWT,ToGo,Registered,10 pc Spicy Wings,Ranch Dip - Regular,Veggie Sticks Spicy
7,908406400,2513,2494847505,Digital,WWT,ToGo,Registered,Ranch Dip - Regular,Add 5 Spicy Wings,8 pc Spicy Wings Combo
8,680349080,820,9679605865,Digital,WWT,ToGo,Guest,Fried Corn - Regular,Chicken Sub,Ranch Dip - Regular
9,123249698,949,8053157007,Digital,WWT,ToGo,Registered,20 pc Grilled Wings,Ranch Dip - Regular,Regular Buffalo Fries


In [81]:
print(f"test data question shape : {test_data_question.shape}")

test data question shape : (1000, 10)


In [82]:
print(test_data_question.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   CUSTOMER_ID            1000 non-null   int64 
 1   STORE_NUMBER           1000 non-null   int64 
 2   ORDER_ID               1000 non-null   int64 
 3   ORDER_CHANNEL_NAME     1000 non-null   object
 4   ORDER_SUBCHANNEL_NAME  1000 non-null   object
 5   ORDER_OCCASION_NAME    1000 non-null   object
 6   CUSTOMER_TYPE          1000 non-null   object
 7   item1                  1000 non-null   object
 8   item2                  1000 non-null   object
 9   item3                  1000 non-null   object
dtypes: int64(3), object(7)
memory usage: 78.3+ KB
None


In [83]:
print(test_data_question.isnull().sum())

CUSTOMER_ID              0
STORE_NUMBER             0
ORDER_ID                 0
ORDER_CHANNEL_NAME       0
ORDER_SUBCHANNEL_NAME    0
ORDER_OCCASION_NAME      0
CUSTOMER_TYPE            0
item1                    0
item2                    0
item3                    0
dtype: int64


In [84]:
print(test_data_question.isnull().sum().sum())

0


In [85]:
print(test_data_question.duplicated().sum())

0


In [86]:
print(test_data_question.nunique())

CUSTOMER_ID              1000
STORE_NUMBER               38
ORDER_ID                 1000
ORDER_CHANNEL_NAME          1
ORDER_SUBCHANNEL_NAME       1
ORDER_OCCASION_NAME         2
CUSTOMER_TYPE               3
item1                      75
item2                      79
item3                      79
dtype: int64
